In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv("temp_data.csv")

In [3]:
df.head(20)

,SentAt,StartedAt,Control,Sad,Intrusion,Encourage,FunThings,IgotThis,Company,Alone,...,Out,Eaten,Avoid,Useful,Enjoy,Pleas,Pleasant,Unpl,Unpleasant,Remarks
0,15/03/2017 16:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/03/2017 09:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16/03/2017 15:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16/03/2017 15:30,16/03/2017 15:30,70.00,42.02,76.74,59.38,35.54,81.14,0.0,24.92,...,66.63,61.97,20.52,41.50,79.85,1.0,50.83,1.0,46.68,NaN
4,16/03/2017 21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,17/03/2017 09:00,17/03/2017 09:17,17.53,18.74,16.93,81.39,82.59,82.19,1.0,16.93,...,87.41,69.84,2.17,87.11,77.77,1.0,88.92,0.0,NaN,NaN
6,17/03/2017 15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,17/03/2017 21:00,17/03/2017 21:05,26.87,27.07,30.18,69.34,54.88,45.44,1.0,19.64,...,62.31,31.39,2.17,50.96,54.48,1.0,73.56,1.0,71.65,NaN
8,18/03/2017 09:00,18/03/2017 09:11,4.78,2.27,10.00,88.62,77.77,79.28,1.0,0.00,...,1.97,51.16,2.87,22.05,70.84,1.0,65.52,0.0,NaN,NaN
9,18/03/2017 15:00,18/03/2017 15:07,1.46,0.00,2.47,82.89,82.69,82.79,1.0,3.07,...,84.00,82.69,1.27,59.60,84.30,1.0,88.31,0.0,NaN,NaN


In [4]:
print(df.shape)
print(df.isna().sum())
print(df.columns)

(1235, 27)
SentAt             0
StartedAt        691
Control          691
Sad              691
Intrusion        691
Encourage        691
FunThings        691
IgotThis         691
Company          691
Alone            691
Thoughts         691
Safe             691
Appointments     691
Cancel           691
Laying           691
Sleep            691
Qualitysleep    1072
Out              691
Eaten            691
Avoid            691
Useful           691
Enjoy            691
Pleas            691
Pleasant        1076
Unpl             691
Unpleasant      1211
Remarks         1087
dtype: int64
Index(['SentAt', 'StartedAt', 'Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Qualitysleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Pleasant', 'Unpl',
       'Unpleasant', 'Remarks'],
      dtype='object')


In [5]:
df = df.drop(columns = ['StartedAt', 'Qualitysleep', 'Pleasant', 'Unpleasant', 'Remarks'])

In [6]:
#drop all observations hich have all missing except for SentAt
to_drop =  ['Control', 'Sad', 'Intrusion', 'Encourage',
       'FunThings', 'IgotThis', 'Company', 'Alone', 'Thoughts', 'Safe',
       'Appointments', 'Cancel', 'Laying', 'Sleep', 'Out',
       'Eaten', 'Avoid', 'Useful ', 'Enjoy', 'Pleas', 'Unpl']

df = df.dropna(subset = to_drop, how = 'any', axis=0)

In [7]:
#make idvar
df['id_var'] = 1
#make date
df['Date'] = pd.to_datetime(df['SentAt'], format = '%d/%m/%Y %H:%M')
#get the date and hour
df['Date_2'] = [d.strftime("%x") for d in df['Date']]
df['Hour'] = [d.strftime("%H") for d in df['Date']]
#make the beepvar required for graphicalvar
df['first_obs'] = [1 if (x == '09' or x == '10') else 0 for x in df['Hour']]
df['second_obs'] = [2 if (x == '15' or x == '16') else 0 for x in df['Hour']]
df['third_obs'] = [3 if (x == '21' or x == '22') else 0 for x in df['Hour']]
df['beep_var'] = df['first_obs'] + df['second_obs'] + df['third_obs']
#order by date
df = df.sort_values(by='Date')

In [8]:
#make day_var
day_var = pd.DataFrame(pd.date_range(df['Date'].min(), df['Date'].max()))
day_var.columns = ['Date_Range']
day_var['day_var'] = [d.strftime("%x") for d in day_var['Date_Range']]
day_var = day_var.drop('Date_Range', axis = 1).reset_index()
df = df.merge(day_var, how = 'left', left_on = 'Date_2', right_on = 'day_var')

In [9]:
#clean by dropping unnecessary columns and renaming columns
df = df.drop(['first_obs', 'second_obs', 'third_obs', 'day_var'], axis = 1)
#rename index to day_var
df = df.rename(columns = {'index': 'day_var'})
#because R doesn't start from -
df['day_var'] = df['day_var']  + 1
#investigate
df['consecutive'] = df['day_var'].diff()
#conver variables to string
df['day_var'] = df['day_var'].astype(str)
df['id_var'] = df['id_var'].astype(str)
df['beep_var'] = df['beep_var'].astype(str)

In [10]:
df['consecutive'].value_counts()

1.0    271
0.0    215
2.0     43
3.0      9
4.0      3
6.0      1
5.0      1
Name: consecutive, dtype: int64

In [11]:
check = df['consecutive'] > 1
check.cumsum().head(50)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
Name: consecutive, dtype: int64

In [12]:
df[df['consecutive'] < 1]

,SentAt,Control,Sad,Intrusion,Encourage,FunThings,IgotThis,Company,Alone,Thoughts,...,Enjoy,Pleas,Unpl,id_var,Date,Date_2,Hour,beep_var,day_var,consecutive
2,17/03/2017 21:00,26.87,27.07,30.18,69.34,54.88,45.44,1.0,19.64,46.15,...,54.48,1.0,1.0,1,2017-03-17 21:00:00,03/17/17,21,3,2,0.0
4,18/03/2017 15:00,1.46,0.00,2.47,82.89,82.69,82.79,1.0,3.07,6.29,...,84.30,1.0,0.0,1,2017-03-18 15:00:00,03/18/17,15,2,3,0.0
5,18/03/2017 21:00,10.20,10.00,12.31,88.01,77.67,80.58,1.0,4.78,5.88,...,76.87,1.0,0.0,1,2017-03-18 21:00:00,03/18/17,21,3,3,0.0
7,19/03/2017 21:00,9.70,9.10,10.90,81.39,78.78,79.28,1.0,4.88,2.47,...,50.36,1.0,0.0,1,2017-03-19 21:00:00,03/19/17,21,3,4,0.0
9,20/03/2017 15:00,13.62,15.42,16.03,76.87,75.36,68.13,0.0,10.90,8.29,...,80.18,1.0,0.0,1,2017-03-20 15:00:00,03/20/17,15,2,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,02/04/2018 22:00,16.93,24.01,27.47,83.19,27.17,31.54,1.0,8.49,17.53,...,45.84,0.0,0.0,1,2018-04-02 22:00:00,04/02/18,22,3,383,0.0
529,08/04/2018 22:00,27.92,31.69,50.96,73.86,10.60,10.45,0.0,39.07,54.58,...,22.80,0.0,0.0,1,2018-04-08 22:00:00,04/08/18,22,3,389,0.0
533,12/04/2018 16:00,33.19,32.74,54.05,82.89,28.68,23.71,1.0,45.54,72.35,...,37.61,1.0,0.0,1,2018-04-12 16:00:00,04/12/18,16,2,393,0.0
536,15/04/2018 16:00,60.90,35.60,56.69,79.58,20.84,21.11,1.0,79.88,68.43,...,27.47,0.0,0.0,1,2018-04-15 16:00:00,04/15/18,16,2,396,0.0


In [13]:
df['day_var'].head(50)

0      1
1      2
2      2
3      3
4      3
5      3
6      4
7      4
8      5
9      5
10     5
11     6
12     6
13     7
14     7
15     8
16     8
17     9
18     9
19     9
20    10
21    10
22    10
23    11
24    11
25    11
26    13
27    13
28    13
29    14
30    14
31    14
32    15
33    15
34    15
35    16
36    16
37    17
38    17
39    18
40    18
41    19
42    19
43    19
44    20
45    20
46    20
47    21
48    21
49    22
Name: day_var, dtype: object

In [14]:
df.to_csv('ESM_using.csv')

In [17]:
df.dtypes

SentAt                  object
Control                float64
Sad                    float64
Intrusion              float64
Encourage              float64
FunThings              float64
IgotThis               float64
Company                float64
Alone                  float64
Thoughts               float64
Safe                   float64
Appointments           float64
Cancel                 float64
Laying                 float64
Sleep                  float64
Out                    float64
Eaten                  float64
Avoid                  float64
Useful                 float64
Enjoy                  float64
Pleas                  float64
Unpl                   float64
id_var                  object
Date            datetime64[ns]
Date_2                  object
Hour                    object
beep_var                object
day_var                 object
consecutive            float64
dtype: object

In [ ]:
print(df.shape)
print(df.isna().sum().sum())
print(df.columns)

In [ ]:

#first = 16/03/2017
#last = 27/04/2018
# just over a year in scope
# note that we have 544 obs but not evenly spaces out

In [ ]:
df.head()